In [7]:
import pandas as pd
import pymorphy2

from tqdm import tqdm
tqdm.pandas()

from nltk.tokenize import word_tokenize
import re

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import GridSearchCV
# import eli5

# +381 62 124 9007
# rocketlady

In [8]:
df = pd.read_csv('data.csv', encoding="utf8", sep='\t')

In [9]:
df['sentiment'].value_counts()

sentiment
negative    30000
neautral    30000
positive    30000
Name: count, dtype: int64

In [10]:
# фильтруем создаем новый DataFrame, который содержит только те строки из df, где значение в столбце 'sentiment' не равно 'neautral'.
df = df[df['sentiment'] != 'neautral']

In [11]:
# Чисто целочисленная индексация
df.iloc[0]['review']

'качество плохое пошив ужасный (горловина наперекос) Фото не соответствует Ткань ужасная рисунок блеклый маленький рукав не такой УЖАС!!!!! не стоит за такие деньги г.......'

In [12]:
# убрали знаки препинания
df['review_processed'] = df['review'].apply(lambda x: re.sub(r'[^\w\s]', '', x)).values

In [13]:
# привели к нижнему регистру
df['review_processed'] = df['review_processed'].apply(lambda x: x.lower())

print(df['review_processed'][0] )

качество плохое пошив ужасный горловина наперекос фото не соответствует ткань ужасная рисунок блеклый маленький рукав не такой ужас не стоит за такие деньги г


In [14]:
# разабъем слова
import nltk
nltk.download('punkt')

df['review_processed'] = df['review_processed'].progress_apply(lambda x: word_tokenize(x, language="russian"))

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\emely\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
100%|██████████| 60000/60000 [00:05<00:00, 10224.95it/s]


In [15]:
df['review_processed'].iloc[0]

['качество',
 'плохое',
 'пошив',
 'ужасный',
 'горловина',
 'наперекос',
 'фото',
 'не',
 'соответствует',
 'ткань',
 'ужасная',
 'рисунок',
 'блеклый',
 'маленький',
 'рукав',
 'не',
 'такой',
 'ужас',
 'не',
 'стоит',
 'за',
 'такие',
 'деньги',
 'г']

In [17]:
morph = pymorphy2.MorphAnalyzer()

In [18]:
morph.parse("сделали")[0].normal_form

'сделать'

In [19]:
df['review_lemmatized'] = df['review_processed'].progress_apply(lambda x: [morph.parse(w)[0].normal_form for w in x])

100%|██████████| 60000/60000 [02:39<00:00, 377.18it/s]


In [20]:
vectorizer = TfidfVectorizer(ngram_range=(1, 1)) 

In [21]:
X = vectorizer.fit_transform(df['review_lemmatized'].apply(lambda x: ' '.join(x)))

In [22]:
X.shape

(60000, 51549)

In [23]:
X_train, X_test, y_train, y_test = train_test_split(
    X, df['sentiment'], test_size=0.3, random_state=42
)

1. train_test_split <br> 2. LogisticRegression <br> 3. fit & predict <br> 4. roc_auc_score

In [24]:
logit = LogisticRegression()

In [25]:
logit.fit(X_train, y_train)

LogisticRegression()

In [26]:
y_pred = logit.predict_proba(X_test)[:, 1]

In [27]:
roc_auc_score(y_test, y_pred)

0.9735222897024753